In [1]:
from pycalphad import Database, variables as v
from pycalphad.variables import Species

dbf = Database('AlCoCrNi-2016Liu-minified.TDB')

In [2]:
import itertools
import string
import sympy
import numpy as np

def genericize(mapping, const_array):
    new_array = []
    for subl in const_array:
        new_array.append(tuple(mapping[x] for x in subl))
    return tuple(new_array)
def sigfigs(x, n):
    """Round x to n significant digits"""
    if x != 0:
        return np.around(x, -(np.floor(np.log10(np.abs(x)))).astype(np.int) + (n - 1))
    else:
        return x
def average_piecewise(pw):
    return sum(x.expr for x in pw.args)/len(pw.args)

def categorize_phase(phase_name, constituents, site_ratios):
    if phase_name == 'LIQUID' and len(constituents) == 1:
        return 'liquid'
    elif phase_name == 'GAS' and len(constituents) == 1:
        return 'gas'
    elif len(constituents) == 1 or \
        (len(constituents) == 2 and len(constituents[1]) == 1 and list(constituents[1])[0] == Species('VA')):
        return 'solid'
    elif all(len(x) == 1 for x in constituents):
        return 'stoichiometric'
    else:
        return 'complex'

def extract_coefficients(expr, polys):
    result = {}
    for poly in polys:
        if poly == 1:
            continue
        else:
            coef = float(expr.coeff(poly))
        result[poly] = coef
        expr = expr - poly*coef
    expr = expr.n()
    if isinstance(expr, (sympy.Float, sympy.Integer)):
        result[1] = float(expr)
        expr = sympy.S.Zero
    if expr is not sympy.S.Zero:
        raise ValueError(expr)
    return tuple(result[p] for p in polys)

total_results = []

for combo in itertools.combinations(dbf.elements - {'VA', '/-'}, 2):
    combo_str = sorted(set(combo) | {'VA'})
    combo = [Species(x) for x in combo_str]
    combo_dict = dict(zip([c for c in combo if c.number_of_atoms > 0], string.ascii_uppercase))
    combo_dict[Species('VA')] = 'VA'
    allowed_phases = set()
    # TODO: Categorize each phase? solid solution, intermetallic, complex intermetallic, liquid, etc.
    for phase_name, phase_obj in dbf.phases.items():
        include_phase = True
        for sublattice in phase_obj.constituents:
            if len(set(sublattice).intersection(combo)) == 0:
                # None of the components in a sublattice are active
                # We cannot build a model of this phase
                include_phase = False
                break
        if include_phase:
            allowed_phases |= {phase_name}
    for param in dbf._parameters.all():
        if param['phase_name'] in allowed_phases and param['parameter_type'] in ['G', 'L']:
            active_pure_elements = [list(x.constituents.keys()) for subl in param['constituent_array'] for x in subl]
            active_pure_elements = [el.upper() for constituents in active_pure_elements for el in constituents]
            active_pure_elements = set(active_pure_elements)
            if not active_pure_elements.issubset(combo_str):
                continue
            generic_combo = [Species(x) for x in sorted(active_pure_elements)]
            gen_combo_dict = dict(zip([c for c in generic_combo if c.number_of_atoms > 0], string.ascii_uppercase))
            gen_combo_dict[Species('VA')] = 'VA'
            gen_combo_dict[Species('*')] = '*'
            ratio_sum = sum(dbf.phases[param['phase_name']].sublattices)
            reduced_ratios = tuple(sigfigs(c, 3) for c in dbf.phases[param['phase_name']].sublattices)
            param_val = param['parameter'].subs(dbf.symbols).subs(dbf.symbols).subs(dbf.symbols).subs(dbf.symbols)
            # If parameter is Piecewise, average it (for now)
            param_val = sympy.piecewise_fold(param_val)
            pw_atoms = param_val.atoms(sympy.Piecewise)
            pw_dict = {key: average_piecewise(key) for key in pw_atoms}
            param_val = param_val.subs(pw_dict)
            # Cross terms must be listed first
            polys = [v.T*sympy.log(1.0e-5*v.P)] + [v.T*sympy.log(v.T), sympy.log(v.T)] + [v.T**n for n in range(-30,0)] + [v.T**n for n in range(1,30)] + [1]
            columns = ['system', 'phase_name', 'phase_type', 'site_ratios',
                       'parameter_order', 'constituent_array', 'parameter_value'] + \
                        [str(x) for x in polys]
            try:
                id_tuple = (tuple(c for c in combo_str if c != 'VA'), param['phase_name'],
                            categorize_phase(param['phase_name'], dbf.phases[param['phase_name']].constituents, reduced_ratios),
                            reduced_ratios, param['parameter_order'],
                            genericize(gen_combo_dict, param['constituent_array']),
                            param_val) + extract_coefficients(param_val, polys)
                total_results.append(id_tuple)
            except ValueError as e:
                print(e)

102862.903225806*((4.65e-11*P + 1.0)**0.8 - 1.0)*exp(9.2e-9*T**2 + 1.7e-5*T) - 1115.9095
172788.461538462*((2.6e-11*P + 1.0)**0.8 - 1.0)*exp(9.2e-9*T**2 + 1.7e-5*T) - 3645.714
172788.461538462*((2.6e-11*P + 1.0)**0.8 - 1.0)*exp(9.2e-9*T**2 + 1.7e-5*T) - 7287.714
102862.903225806*((4.65e-11*P + 1.0)**0.8 - 1.0)*exp(9.2e-9*T**2 + 1.7e-5*T) - 1115.9095
172788.461538462*((2.6e-11*P + 1.0)**0.8 - 1.0)*exp(9.2e-9*T**2 + 1.7e-5*T) - 3645.714
172788.461538462*((2.6e-11*P + 1.0)**0.8 - 1.0)*exp(9.2e-9*T**2 + 1.7e-5*T) - 7287.714
102862.903225806*((4.65e-11*P + 1.0)**0.8 - 1.0)*exp(9.2e-9*T**2 + 1.7e-5*T) - 1115.9095
172788.461538462*((2.6e-11*P + 1.0)**0.8 - 1.0)*exp(9.2e-9*T**2 + 1.7e-5*T) - 3645.714
172788.461538462*((2.6e-11*P + 1.0)**0.8 - 1.0)*exp(9.2e-9*T**2 + 1.7e-5*T) - 7287.714


In [3]:
import pandas as pd
from IPython.display import display

df = pd.DataFrame(total_results, columns=columns)
for p in polys:
    df[str(p)] = pd.to_numeric(df[str(p)])
multi_df = df.set_index(['phase_type', 'site_ratios', 'constituent_array', 'parameter_order'])
display(multi_df)

system  \
phase_type     site_ratios       constituent_array     parameter_order             
solid          (1.0, 3.0)        ((VA,), (VA,))        0                (CR, NI)   
                                 ((A, VA), (VA,))      0                (CR, NI)   
               (1.0, 0.5)        ((A,), (VA,))         0                (CR, NI)   
complex        (0.5, 0.5, 3.0)   ((A,), (VA,), (VA,))  0                (CR, NI)   
                                 ((VA,), (A,), (VA,))  0                (CR, NI)   
liquid         (1.0,)            ((A,),)               0                (CR, NI)   
solid          (1.0, 1.0)        ((A,), (VA,))         0                (CR, NI)   
               (1.0, 3.0)        ((A,), (VA,))         0                (CR, NI)   
                                 ((A, VA), (VA,))      0                (CR, NI)   
               (1.0, 0.5)        ((A,), (VA,))         0                (CR, NI)   
complex        (0.5, 0.5, 3.0)   ((VA,), (A,), (VA,))  0                (CR, NI)   
                                 ((A,), (VA,), (VA,))  0                (CR, NI)   
liquid         (1.0,)            ((A, B),)             0                (CR, NI)   
                                                       1                (CR, NI)   
solid          (1.0, 1.0)        ((A, B), (VA,))       0                (CR, NI)   
                                                       1                (CR, NI)   
               (1.0, 3.0)        ((A, B), (VA,))       0                (CR, NI)   
                                                       1                (CR, NI)   
complex        (0.5, 0.5, 3.0)   ((A,), (B,), (VA,))   0                (CR, NI)   
                                 ((B,), (A,), (VA,))   0                (CR, NI)   
               (0.75, 0.25, 1.0) ((B,), (A,), (VA,))   0                (CR, NI)   
                                 ((A,), (B,), (VA,))   0                (CR, NI)   
                                 ((A, B), (A,), (VA,)) 0                (CR, NI)   
                                 ((A, B), (B,), (VA,)) 0                (CR, NI)   
                                 ((A, B), (A,), (VA,)) 1                (CR, NI)   
                                 ((A, B), (B,), (VA,)) 1                (CR, NI)   
                                 ((*,), (A, B), (VA,)) 0                (CR, NI)   
                                                       1                (CR, NI)   
                                 ((A, B), (*,), (VA,)) 0                (CR, NI)   
                                                       1                (CR, NI)   
...                                                                          ...   
solid          (1.0, 1.0)        ((A,), (VA,))         0                (AL, CO)   
               (1.0, 3.0)        ((A,), (VA,))         0                (AL, CO)   
                                 ((A, VA), (VA,))      0                (AL, CO)   
               (1.0, 0.5)        ((A,), (VA,))         0                (AL, CO)   
complex        (0.5, 0.5, 3.0)   ((A,), (VA,), (VA,))  0                (AL, CO)   
                                 ((VA,), (A,), (VA,))  0                (AL, CO)   
liquid         (1.0,)            ((A, B),)             0                (AL, CO)   
                                                       1                (AL, CO)   
                                                       2                (AL, CO)   
                                                       3                (AL, CO)   
solid          (1.0, 1.0)        ((A, B), (VA,))       0                (AL, CO)   
                                                       2                (AL, CO)   
               (1.0, 3.0)        ((A, B), (VA,))       0                (AL, CO)   
complex        (0.5, 0.5, 3.0)   ((A,), (B,), (VA,))   0                (AL, CO)   
                                 ((B,), (A,), (VA,))   0                (AL, CO)   
               (0.75, 0.25, 1.0) ((B,), (A

In [4]:
for system_name, system_df in df.groupby('system'):
    system_df = system_df.drop_duplicates(subset='phase_name')
    print('-')
    print('-'.join(system_name))
    print(system_df['phase_type'].value_counts().to_string())

-
AL-CO
stoichiometric    4
solid             3
complex           3
liquid            1
-
AL-CR
stoichiometric    8
complex           4
solid             3
liquid            1
-
AL-NI
solid             3
complex           3
stoichiometric    2
liquid            1
-
CO-CR
solid      3
complex    3
liquid     1
-
CO-NI
solid      3
complex    2
liquid     1
-
CR-NI
solid      3
complex    3
liquid     1


In [5]:
import numpy as np
multi_df.groupby(level=[0,1,2,3]).agg(['mean', 'std', 'count'])

T*log(1.0e-5*P)  \
                                                                                  mean   
phase_type     site_ratios       constituent_array     parameter_order                   
complex        (0.5, 0.5, 3.0)   ((A,), (B,), (VA,))   0                           0.0   
                                 ((A,), (VA,), (VA,))  0                           0.0   
                                 ((B,), (A,), (VA,))   0                           0.0   
                                 ((VA,), (A,), (VA,))  0                           0.0   
               (0.75, 0.25, 1.0) ((*,), (A, B), (VA,)) 0                           0.0   
                                                       1                           0.0   
                                 ((A,), (B,), (VA,))   0                           0.0   
                                 ((A, B), (*,), (VA,)) 0                           0.0   
                                                       1                           0.0   
                                 ((A, B), (A,), (VA,)) 0                           0.0   
                                                       1                           0.0   
                                 ((A, B), (B,), (VA,)) 0                           0.0   
                                                       1                           0.0   
                                 ((B,), (A,), (VA,))   0                           0.0   
               (3.0, 2.0, 1.0)   ((A,), (*,), (B, VA)) 0                           0.0   
                                 ((A,), (A,), (B,))    0                           0.0   
                                 ((A,), (A,), (VA,))   0                           0.0   
                                 ((A,), (A, B), (*,))  0                           0.0   
                                 ((A,), (B,), (B,))    0                           0.0   
                                 ((A,), (B,), (VA,))   0                           0.0   
               (8.0, 18.0, 4.0)  ((A,), (A,), (B,))    0                           0.0   
                                 ((A,), (B,), (B,))    0                           0.0   
                                 ((B,), (A,), (A,))    0                           0.0   
                                 ((B,), (B,), (A,))    0                           0.0   
liquid         (1.0,)            ((A,),)               0                           0.0   
                                 ((A, B),)             0                           0.0   
                                                       1                           0.0   
                                                       2                           0.0   
                                                       3                           0.0   
                                                       4                           0.0   
solid          (1.0, 0.5)        ((A,), (VA,))         0                           0.0   
                                 ((A, B), (VA,))       0                           0.0   
               (1.0, 1.0)        ((A,), (VA,))         0                           0.0   
                                 ((A, B), (VA,))       0                           0.0   
                                                       1                           0.0   
                                                       2                           0.0   
                                                       3                           0.0   
               (1.0, 3.0)        ((A,), (VA,))         0                           0.0   
                                 ((A, B), (VA,))       0                           0.0   
                                                       1                           0.0   
                                 ((A, VA), (VA,))      0                           0.0   
                                 ((VA,), (VA,))        0                           0.0   
stoichiometric (0.375